# ANN model implementation on the Pre-processed Data:

In [117]:
import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
%matplotlib inline

In [118]:
X = pd.read_csv('X_data.csv')
y = pd.read_csv('y_data.csv')
X = X.drop("Unnamed: 0",axis=1)
y = y.drop("Unnamed: 0",axis=1)
X.index = X.index + 1
y.index = y.index + 1
X.shape,y.shape

((10001, 12), (10001, 1))

In [119]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
1,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.880,0.0,0.0,1.0
2,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.580,1.0,0.0,0.0
3,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.570,1.0,0.0,0.0
4,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.630,0.0,0.0,1.0
5,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.100,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9997,516.0,1.0,35.0,10.0,57369.61,1.0,1.0,1.0,101699.770,1.0,0.0,0.0
9998,709.0,0.0,36.0,7.0,0.00,1.0,0.0,1.0,42085.580,0.0,1.0,0.0
9999,772.0,1.0,42.0,3.0,75075.31,2.0,1.0,0.0,92888.520,1.0,0.0,0.0
10000,792.0,0.0,28.0,4.0,130142.79,1.0,1.0,0.0,38190.780,1.0,0.0,0.0


In [120]:
y

,Exited
1,1.0
2,0.0
3,1.0
4,0.0
5,0.0
...,...
9997,0.0
9998,1.0
9999,1.0
10000,0.0


## Train Test Splitting:

In [121]:
from sklearn.model_selection import train_test_split

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [123]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((8000, 12), (2001, 12), (8000, 1), (2001, 1))

## Standardizing the Training and Testing Input Data:

In [124]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [125]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Saving the Standard Scale instance with Pickle:

In [126]:
if os.path.exists("scaler.pkl"):
    print("File already exist!")
else:
    with open("scaler.pkl","wb") as f:
        pickle.dump(scaler,f)
        print("File created and 'scaler.pkl' saved.")

File already exist!


# Implementation of ANN Model:

## Importing Libraries for Model:

In [127]:
import keras
from keras.models import Sequential # Model's Instance
from keras.layers import Dense # Hidden layers inside models

### Read for the Classes:
1. Sequential - "https://keras.io/guides/sequential_model/"
2. Dense Layer - "https://keras.io/api/layers/core_layers/dense/"

## Creating the Model:

In [128]:
# Input layer's should contain how many neurons:
X_train.shape[1]

12

In [129]:
model = Sequential(
    [
        Dense(units=64,activation='relu',input_shape=(X_train.shape[1],)), # Hidden Layer 1: Connected with input layers.
        Dense(units=32,activation='relu'), # Hidden Layer 2: No need to provide the input size again
        Dense(units=1,activation='sigmoid'), # Output Layer
    ]
)

d:\Multimedia\Programming\DeepLearning\Projects\01simpleANN\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [130]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

## Compiling the Model:

### Optimizers:

In [131]:
# A way to Define the optimizer and loss function
opt = keras.optimizers.Adam(learning_rate=0.01)
losses = keras.losses.BinaryCrossentropy() 

In [132]:
# Compiling the Model:
model.compile(optimizer=opt,loss=losses,metrics=["accuracy"]) #type:ignore

# Setting up the TensorBoard:
TensorBoard helps to visualize all our logs while training our model.

In [133]:
import datetime # To generate a record of logs while training
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
from keras.callbacks import TensorBoard
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

## Setting up the Early Stopping:

In [134]:
from keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

# Trainig the ANN Model:

In [135]:
history = model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8301 - loss: 0.4069 - val_accuracy: 0.8496 - val_loss: 0.3792
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8512 - loss: 0.3656 - val_accuracy: 0.8506 - val_loss: 0.3666
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8509 - loss: 0.3585 - val_accuracy: 0.8531 - val_loss: 0.3513
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8534 - loss: 0.3592 - val_accuracy: 0.8486 - val_loss: 0.3599
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8549 - loss: 0.3536 - val_accuracy: 0.8561 - val_loss: 0.3662
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8559 - loss: 0.3539 - val_accuracy: 0.8571 - val_loss: 0.3542
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8571 - loss: 0.3498 - val_accuracy: 0.8441 - val_loss: 0.3604
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8550 - loss: 0.3462 - val_accu

In [136]:
if os.path.exists('model.h5'):
    print("File already created and saved.")
else:
    model.save('model.h5')
    print("File created and saved.")

File already created and saved.


# Loading TensorBoard Extension:

In [137]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [139]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6009 (pid 14816), started 0:00:09 ago. (Use '!kill 14816' to kill it.)